In [1]:
from ipynb.fs.full.handlers import input_1, input_2, input_3

uploaded_file_name_temp = input_1(); 
output_file_name_temp = input_2(); 
region_temp = input_3()


Please provide the name of file you uploaded (in csv):


Text(value='', placeholder='eg. app_bundle.csv')

Please provide the desired name of the output file (in csv):


Text(value='', placeholder='eg. output.csv')

Please select the region:


RadioButtons(options=('US', 'CA', 'EMEA', 'LATAM', 'APAC'), value='US')

Button(description='Submit and Run', style=ButtonStyle())

In [4]:
import pandas as pd
import requests
import json
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, DecimalType, IntegerType
from pyspark.sql.window import Window
from pyspark.sql import SparkSession,SQLContext
spark = SparkSession.builder.appName("app_store_url_generator").getOrCreate()
sc=spark.sparkContext
sqlContext = SQLContext(sc)

def main_func():
    print("Sit back and relax. The script is running...\n")
    df = spark.createDataFrame(pd.read_csv(uploaded_file_name, header=None))
    ab_list = df.distinct().rdd.flatMap(lambda x: x).collect()

    # Creating country list as per the user region input
    if region == "US": 
        country_list = ["us"]
        country_name_list = ["US"]
    elif region == "CA": 
        country_list = ["ca"]
        country_name_list = ["CA"]
    elif region == "EMEA": 
        country_list = ["gb","fr","es","it","de"]
        country_name_list = ["UK","France","Spain","Italy","Germany"]
    elif region == "LATAM":
        country_list = ["br","mx","ar","cl","co"]
        country_name_list = ["Brazil","Mexico","Argentina","Chile","Colombia"]
    elif region == "APAC": 
        country_list = ["au","sg","in","jp","cn"]
        country_name_list = ["Australia","Singapore","India","Japan","China"]

    # for LGE app bundles

    print("Loading LGE app bundles for the selected region...")
    url = "https://us.lgappstv.com/api/common/retrieveNation.ajax"
    response = requests.request("POST", url)
    obj = json.loads(response.text)
    country_list_temp = []
    for cnt in obj["cntryList"]:
        if cnt["cntryCode"] == "C01" and "cn" in country_list: country_list_temp.append("wasu")
        elif cnt["cntryCode"].lower() in country_list: country_list_temp.append(cnt["cntryCode"].lower())
        else: continue
    country_list_temp = [*set(country_list_temp)]
    lge_app_store_url_list = []
    lge_app_bundle_list = []
    for country in country_list_temp:
        url = "https://" + country + ".lgappstv.com/api/tvapp/retrieveAllAppList.ajax"
        payload={'rowCount': '200'}
        response = requests.request("POST", url, data=payload)
        obj = json.loads(response.text)
        for app in obj["allAppList"]:
            url = "https://" + country + ".lgappstv.com/main/tvapp/detail?appId=" + str(app["appId"])
            lge_app_store_url_list.append(url)
            lge_app_bundle_list.append(str(app["appId"]))
    lge_app_bundle_list = [*set(lge_app_bundle_list)]
    print("Successfully loaded all the LGE app bundles.\n")

    # Generating app bundles for selected Region

    df_input = sqlContext.createDataFrame(zip(ab_list), schema=['app_bundle'])
    i = 0

    for country in country_list:

        print("\nGenerating app store urls for country: ", country_name_list[i],"\n")

        android = "https://play.google.com/store/apps/details?id=" # starts or ends with "com"
        amazon = "https://www.amazon.com/dp/" # starts with b or B
        microsoft = "https://apps.microsoft.com/store/detail/" # starts with "9"
        samsung = "https://www.samsung.com/"+ country + "/appstore/app/" # starts with "g" or "G"
        playstation = "https://store.playstation.com/en-"+ country + "/product/" # starts with "UP" or "up"
        vizio = "https://www.vizio.com/en/smart-tv-apps?appId=" # starts with "vizio"
        roku = "https://channelstore.roku.com/details/" # should not redirect to https://channelstore.roku.com/browse
        if country == "cn": lge = "https://"+ "wasu" + ".lgappstv.com/main/tvapp/detail?appId=" # should be present in the LGE content store
        else: lge = "https://"+ country + ".lgappstv.com/main/tvapp/detail?appId=" # should be present in the LGE content store
        ios = "https://apps.apple.com/"+ country + "/app/id" # part of corner case

        app_bundle_list = []
        app_store_url_list = []
        device_brand_list = []
        status_code_list = []

        j = 1
        for app_bundle in ab_list:
            print(j, " Generating app store url for app bundle:", app_bundle)
            j+=1
            app_store_url = ""
            device_brand = ""
            if app_bundle is None:
                continue
            r_app_store_url_android = requests.get(android + app_bundle)
            r_app_store_url_ios = requests.get(ios + app_bundle)
            r_app_store_url_roku = requests.get(roku + app_bundle)
            r_app_store_url_lge = requests.get(lge + app_bundle)
            if (app_bundle.startswith("com") or app_bundle.endswith("com")) and r_app_store_url_android.status_code < 400: 
                app_store_url = android + app_bundle
                device_brand = "Android"
            elif r_app_store_url_roku.status_code < 400 and app_bundle.isnumeric() and r_app_store_url_roku.url != "https://channelstore.roku.com/browse": 
                app_store_url = roku + app_bundle
                device_brand = "Roku"
            elif (lge + app_bundle) in lge_app_store_url_list: 
                app_store_url = lge + app_bundle
                device_brand = "LG"
            elif (app_bundle.startswith("b") or app_bundle.startswith("B")) and (app_bundle[1].isnumeric()): 
                app_store_url = amazon + app_bundle
                device_brand = "Amazon"
            elif app_bundle.startswith("9") and app_bundle[1].isalpha():
                app_store_url = microsoft + app_bundle
                device_brand = "Microsoft"
            elif (app_bundle.startswith("g") or app_bundle.startswith("G")) and (app_bundle[1].isnumeric()) and (len(app_bundle) == 12): 
                app_store_url = samsung + app_bundle
                device_brand = "Samsung"
            elif app_bundle.startswith("UP") or app_bundle.startswith("up"): 
                app_store_url = playstation + app_bundle.upper()
                device_brand = "Playstation"
            elif app_bundle.startswith("vizio"):
                app_store_url = vizio + app_bundle
                device_brand = "Vizio"
            elif r_app_store_url_ios.status_code < 400 and app_bundle.isnumeric(): 
                app_store_url = ios + app_bundle
                device_brand = "iOS"
            else:
                app_store_url = "No App store URL found"
                device_brand = "Not Found"
                app_bundle_list.append(app_bundle)
                app_store_url_list.append(app_store_url)
                status_code_list.append(404)
                device_brand_list.append(device_brand)
                continue
            r = requests.head(app_store_url)
            app_bundle_list.append(app_bundle)
            app_store_url_list.append(app_store_url)
            status_code_list.append(r.status_code)
            device_brand_list.append(device_brand)

        col_name_app_store_url = country_name_list[i] + "_app_store_url"
        col_name_device = country_name_list[i] + "_device_brand"
        df_temp = sqlContext.createDataFrame(zip(app_bundle_list, app_store_url_list, device_brand_list), schema=['app_bundle', col_name_app_store_url, col_name_device])
        df_input = df_input.join(df_temp, ["app_bundle"], "inner")
        i+=1
    print("\nSuccessfully generated the app store urls and device for the chosen region.\n")
    print("Storing the data in the output file...")
    df_input.toPandas().to_csv(output_file_name, index=False)
    print("Succefully stored the data in the output file.")

uploaded_file_name = uploaded_file_name_temp.value; output_file_name = output_file_name_temp.value; region = region_temp.value

if uploaded_file_name.endswith(".csv") and output_file_name.endswith(".csv"): main_func()
else: print("The uploaded file name or the desired output file name doesn't end with '.csv', hence, the script terminated.")
    

Sit back and relax. The script is running...

Loading LGE app bundles for the selected region...
Successfully loaded all the LGE app bundles.


Generating app store urls for country:  UK 

1  Generating app store url for app bundle: com.rakutentv.philips
2  Generating app store url for app bundle: samsung.com
3  Generating app store url for app bundle: com.discoveryplus.youview
4  Generating app store url for app bundle: 591991
5  Generating app store url for app bundle: com.sky.solitaire_classic
6  Generating app store url for app bundle: uk.co.netgem.origin
7  Generating app store url for app bundle: b077h71cnx
8  Generating app store url for app bundle: com.roku.cbsnews
9  Generating app store url for app bundle: titan_os
10  Generating app store url for app bundle: g20363016136
11  Generating app store url for app bundle: g20363016142
12  Generating app store url for app bundle: 239514
13  Generating app store url for app bundle: com.wedotv.eutelsat.wedomovies
14  Generating app st

135  Generating app store url for app bundle: empty
136  Generating app store url for app bundle: b07q3t4jwk
137  Generating app store url for app bundle: com.sky.wordlefull
138  Generating app store url for app bundle: com.cbs.ca
139  Generating app store url for app bundle: g15350005739
140  Generating app store url for app bundle: 351189
141  Generating app store url for app bundle: com.five.freeviewplaylg
142  Generating app store url for app bundle: 1233595
143  Generating app store url for app bundle: g21003016186
144  Generating app store url for app bundle: b019dchdzk
145  Generating app store url for app bundle: 1196211
146  Generating app store url for app bundle: com.sportstribal.netgem
147  Generating app store url for app bundle: lgappstv.com
148  Generating app store url for app bundle: 20007319
149  Generating app store url for app bundle: b08zg55rnk
150  Generating app store url for app bundle: com.turner.tnt.android.networkapp
151  Generating app store url for app bund

275  Generating app store url for app bundle: com.sky.crossy_road
276  Generating app store url for app bundle: b07dbbtdbs
277  Generating app store url for app bundle: g18204011569
278  Generating app store url for app bundle: g14363001012
279  Generating app store url for app bundle: com.lg.bestofminecraft
280  Generating app store url for app bundle: com.sky.pixel_dash
281  Generating app store url for app bundle: 456000021
282  Generating app store url for app bundle: com.sky.carrot_mania2
283  Generating app store url for app bundle: com.yupptv.lycatv
284  Generating app store url for app bundle: 51106
285  Generating app store url for app bundle: com.dazn
286  Generating app store url for app bundle: id329913454
287  Generating app store url for app bundle: com.toongoggles.html5.tv.html5.philips
288  Generating app store url for app bundle: vevo
289  Generating app store url for app bundle: 665805393
290  Generating app store url for app bundle: up8835-cusa09505_00-performgroup20

112  Generating app store url for app bundle: jivjixagco.orangetv
113  Generating app store url for app bundle: yupptv.lg
114  Generating app store url for app bundle: com.sky.alice_hearts
115  Generating app store url for app bundle: 1455434920
116  Generating app store url for app bundle: com.rakuten.tv
117  Generating app store url for app bundle: b07yyltrqw
118  Generating app store url for app bundle: b08pzbh9b4
119  Generating app store url for app bundle: b00hev5rme
120  Generating app store url for app bundle: com.sky.burger
121  Generating app store url for app bundle: 3202204027073
122  Generating app store url for app bundle: com.roku.watchfreeflix
123  Generating app store url for app bundle: com.sky.trivia_crack
124  Generating app store url for app bundle: 13535
125  Generating app store url for app bundle: com.sky.bubble_jam
126  Generating app store url for app bundle: b004y1wcde
127  Generating app store url for app bundle: 111299001432
128  Generating app store url fo

244  Generating app store url for app bundle: com.sky.like_nastya_italy
245  Generating app store url for app bundle: com.sky.super_collapse
246  Generating app store url for app bundle: com.xumo.hisense
247  Generating app store url for app bundle: 457877
248  Generating app store url for app bundle: 3201903018105
249  Generating app store url for app bundle: com.orange.es.orangetv
250  Generating app store url for app bundle: g16300008593
251  Generating app store url for app bundle: 1558867879
252  Generating app store url for app bundle: b00ft7lgpm
253  Generating app store url for app bundle: com.viki.viki-ios
254  Generating app store url for app bundle: b07qwrt42d
255  Generating app store url for app bundle: 20445
256  Generating app store url for app bundle: 300620
257  Generating app store url for app bundle: 9nt1rwrl3jnn
258  Generating app store url for app bundle: b004u7bxbo
259  Generating app store url for app bundle: com.rakutentv.chromecast
260  Generating app store ur

...

55  Generating app store url for app bundle: g18229011675
56  Generating app store url for app bundle: 173516
57  Generating app store url for app bundle: 1169805
58  Generating app store url for app bundle: com.paramountplus.virginmedia
59  Generating app store url for app bundle: b00qrzuqqk
60  Generating app store url for app bundle: com.samsung.android.tvplus
61  Generating app store url for app bundle: b01mg2701v
62  Generating app store url for app bundle: 334
63  Generating app store url for app bundle: g18263011896
64  Generating app store url for app bundle: 122834
65  Generating app store url for app bundle: com.viki.kindle
66  Generating app store url for app bundle: com.channel5.demand5
67  Generating app store url for app bundle: g20359016121
68  Generating app store url for app bundle: tv.pluto.android
69  Generating app store url for app bundle: 27424
70  Generating app store url for app bundle: com.sky.rummikub
71  Generating app store url for app bundle: com.einsundein

196  Generating app store url for app bundle: 9nblggh3zzvf
197  Generating app store url for app bundle: com.mobileiq.demand5
198  Generating app store url for app bundle: com.rakutentv.webos
199  Generating app store url for app bundle: vevo.samsungtvplus
200  Generating app store url for app bundle: 256567
201  Generating app store url for app bundle: 1089414
202  Generating app store url for app bundle: 952316
203  Generating app store url for app bundle: com.sky.solitaire_scorpion
204  Generating app store url for app bundle: com.yupptv.androidtv.lebara
205  Generating app store url for app bundle: com.five.freesat
206  Generating app store url for app bundle: 951556
207  Generating app store url for app bundle: 1136580
208  Generating app store url for app bundle: com.rakutentv.hisense
209  Generating app store url for app bundle: b074f28myj
210  Generating app store url for app bundle: 1557287817
211  Generating app store url for app bundle: com.orange.es.stb
212  Generating app 

30  Generating app store url for app bundle: de.exaring.waipu.o2
31  Generating app store url for app bundle: b0bq4z23cf
32  Generating app store url for app bundle: com.bt.chess
33  Generating app store url for app bundle: b0711z67w9
34  Generating app store url for app bundle: tv.atmosphere.atmospheretv.fod
35  Generating app store url for app bundle: com.sky.rider
36  Generating app store url for app bundle: tv.wuaki.wuakitv-player
37  Generating app store url for app bundle: 2595
38  Generating app store url for app bundle: b0cl3nzbym
39  Generating app store url for app bundle: aaajixagco.jazzteltv
40  Generating app store url for app bundle: g15115002089
41  Generating app store url for app bundle: 1087412
42  Generating app store url for app bundle: tinyhousenation
43  Generating app store url for app bundle: b00kdsgipk
44  Generating app store url for app bundle: com.alteox.euronews
45  Generating app store url for app bundle: de.exaring.waipu.samsung
46  Generating app store u

165  Generating app store url for app bundle: com.rakutentv.sony
166  Generating app store url for app bundle: com.five.samsung
167  Generating app store url for app bundle: fr.bouyguestelecom.tv.android
168  Generating app store url for app bundle: uk.co.news.talktv.freeviewplay
169  Generating app store url for app bundle: com.five.freelyhisense
170  Generating app store url for app bundle: 474315
171  Generating app store url for app bundle: 9p9lv240kq9r
172  Generating app store url for app bundle: 658700
173  Generating app store url for app bundle: g20280015643
174  Generating app store url for app bundle: 1340650234
175  Generating app store url for app bundle: tv.molotov.app
176  Generating app store url for app bundle: com.sky.solitaire_towers
177  Generating app store url for app bundle: b07q7n4gzg
178  Generating app store url for app bundle: 232373
179  Generating app store url for app bundle: 20006184
180  Generating app store url for app bundle: com.yupptv.androidtv
181  

297  Generating app store url for app bundle: g00002687241
298  Generating app store url for app bundle: com.sky.khaby_lame

Successfully generated the app store urls and device for the chosen region.

Storing the data in the output file...
Succefully stored the data in the output file.


In [4]:
import pandas as pd
import requests
import json
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, DecimalType, IntegerType
from pyspark.sql.window import Window
from pyspark.sql import SparkSession,SQLContext
spark = SparkSession.builder.appName("app_store_url_generator").getOrCreate()
sc=spark.sparkContext
sqlContext = SQLContext(sc)

def toPySparkDf(data, spark):
    from pyspark.sql import SQLContext
    from pyspark.sql.dataframe import DataFrame
    sqlctx = SQLContext(spark)
    return DataFrame(data, sqlctx)

data = spark._jvm.tv.alphonso.datalib.dal.reader.SSLogsReader().date("2023-09-01", "2023-09-01").read()

df = toPySparkDf(data,spark)

df.filter(col("campaign_id") == "0d8522f9-37dc-431a-ba69-31ae7eeea700").select("ip").distinct().count()

13968

In [15]:
!hdfs dfs -ls /data/springservelogs/aggregated-v2/daily/date=2023-09-05/core_event_type=impression/

Found 5 items
-rw-r--r--   3 alpha supergroup  135663759 2023-09-06 10:21 /data/springservelogs/aggregated-v2/daily/date=2023-09-05/core_event_type=impression/part-00172-79bd2dcb-db55-4537-9298-0621d483cee4.c000.snappy.parquet
-rw-r--r--   3 alpha supergroup  155336757 2023-09-06 10:21 /data/springservelogs/aggregated-v2/daily/date=2023-09-05/core_event_type=impression/part-00175-79bd2dcb-db55-4537-9298-0621d483cee4.c000.snappy.parquet
-rw-r--r--   3 alpha supergroup  174652900 2023-09-06 10:19 /data/springservelogs/aggregated-v2/daily/date=2023-09-05/core_event_type=impression/part-00198-79bd2dcb-db55-4537-9298-0621d483cee4.c000.snappy.parquet
-rw-r--r--   3 alpha supergroup  150027950 2023-09-06 10:20 /data/springservelogs/aggregated-v2/daily/date=2023-09-05/core_event_type=impression/part-00209-79bd2dcb-db55-4537-9298-0621d483cee4.c000.snappy.parquet
-rw-r--r--   3 alpha supergroup  210024838 2023-09-06 10:24 /data/springservelogs/aggregated-v2/daily/date=2023-09-05/core_event_type=

In [2]:
import pandas as pd
import requests
import json
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, DecimalType, IntegerType
from pyspark.sql.window import Window
from pyspark.sql import SparkSession,SQLContext
spark = SparkSession.builder.appName("app_store_url_generator").getOrCreate()
sc=spark.sparkContext
sqlContext = SQLContext(sc)

# Get country list
url = "https://us.lgappstv.com/api/common/retrieveNation.ajax"
response = requests.request("POST", url)
obj = json.loads(response.text)
country_list_temp = []
country_list_name_temp = []
i=0
for cnt in obj["cntryList"]:
    temp = []
    if cnt["cntryCode"] == "C01": temp.append("wasu")
    else: temp.append(cnt["cntryCode"].lower())
    temp.append(cnt["cntryName"])
    if i>0:
        if temp[1] == country_list_temp[-1][1]: continue
    country_list_temp.append(temp)
    i+=1
# print(country_list_temp)

for country in country_list_temp:
    url = "https://"+ country[0] +".lgappstv.com/api/tvapp/retrieveAllAppList.ajax"
    payload={'rowCount': '1'}
    response = requests.request("POST", url, data=payload)
    obj = json.loads(response.text)
    for app in obj["allAppList"]:
        print(app)
#         url = "https://"+ country[0] +".lgappstv.com/main/tvapp/detail?appId=" + str(app["appId"])
#         print(country[1],"@", app["appId"], "@", url, "@", app["appName"], "@", app["catName"])

# Get all app bundles for the country



{'appId': 139, 'appName': 'Buy Palace', 'catName': 'Game', 'catCode': '002', 'dplyDate': '2011-04-15 02:46:20.0', 'downCount': 0, 'appPath': 'http://ngfts.lge.com/fts/gftsDownload.lge?biz_code=APP_STORE&func_code=APP_ICON&file_path=/appstore/app/icon/20110308/11_148x148.png', 'age': '0', 'sellrUsrName': 'UANGEL', 'avgSscr': 5.6, 'sellrUsrNo': 208, 'preFlag': 'N'}
{'appId': 140, 'appName': 'Natural Object', 'catName': 'Game', 'catCode': '002', 'dplyDate': '2011-03-23 01:28:43.0', 'downCount': 0, 'appPath': 'http://ngfts.lge.com/fts/gftsDownload.lge?biz_code=APP_STORE&func_code=APP_ICON&file_path=/appstore/app/icon/20110308/7_148x148.png', 'age': '0', 'sellrUsrName': 'UANGEL', 'avgSscr': 4.4, 'sellrUsrNo': 208, 'preFlag': 'N'}
{'appId': 141, 'appName': 'Puzzle Game', 'catName': 'Game', 'catCode': '002', 'dplyDate': '2011-04-11 04:04:04.0', 'downCount': 0, 'appPath': 'http://ngfts.lge.com/fts/gftsDownload.lge?biz_code=APP_STORE&func_code=APP_ICON&file_path=/appstore/app/icon/20110308/6_14

{'appId': 1205774, 'appName': '4K Player', 'catName': 'Entertainment', 'catCode': '003', 'dplyDate': '2023-06-02 07:50:17.0', 'downCount': 0, 'appPath': 'http://ngfts.lge.com/fts/gftsDownload.lge?biz_code=APP_STORE&func_code=APP_ICON&file_path=/appstore/app/icon/20230222/37799822.png', 'age': '4', 'sellrUsrName': 'zen llc ', 'avgSscr': 4.5, 'sellrUsrNo': 22940615, 'preFlag': 'N'}
{'appId': 1205826, 'appName': '67 Play', 'catName': 'Entertainment', 'catCode': '003', 'dplyDate': '2023-04-11 02:30:42.0', 'downCount': 0, 'appPath': 'http://ngfts.lge.com/fts/gftsDownload.lge?biz_code=APP_STORE&func_code=APP_ICON&file_path=/appstore/app/icon/20230223/37800350.png', 'age': '12', 'sellrUsrName': 'EiTV', 'avgSscr': 6, 'sellrUsrNo': 22939245, 'preFlag': 'N'}
{'appId': 1205917, 'appName': 'etvwin', 'catName': 'Entertainment', 'catCode': '003', 'dplyDate': '2023-08-15 23:27:07.0', 'downCount': 0, 'appPath': 'http://ngfts.lge.com/fts/gftsDownload.lge?biz_code=APP_STORE&func_code=APP_ICON&file_path=

{'appId': 139, 'appName': 'Buy Palace', 'catName': '游戏', 'catCode': '002', 'dplyDate': '2019-10-28 06:10:23.0', 'downCount': 0, 'appPath': 'http://ngfts.lge.com/fts/gftsDownload.lge?biz_code=APP_STORE&func_code=APP_ICON&file_path=/appstore/app/icon/20110308/11_148x148.png', 'age': '0', 'sellrUsrName': 'UANGEL', 'avgSscr': 3.5, 'sellrUsrNo': 208, 'preFlag': 'N'}
{'appId': 140, 'appName': 'Natural Object', 'catName': '游戏', 'catCode': '002', 'dplyDate': '2019-10-28 06:10:05.0', 'downCount': 0, 'appPath': 'http://ngfts.lge.com/fts/gftsDownload.lge?biz_code=APP_STORE&func_code=APP_ICON&file_path=/appstore/app/icon/20110308/7_148x148.png', 'age': '0', 'sellrUsrName': 'UANGEL', 'avgSscr': 5.6, 'sellrUsrNo': 208, 'preFlag': 'N'}
{'appId': 141, 'appName': 'Puzzle Game', 'catName': '游戏', 'catCode': '002', 'dplyDate': '2019-10-28 06:09:48.0', 'downCount': 0, 'appPath': 'http://ngfts.lge.com/fts/gftsDownload.lge?biz_code=APP_STORE&func_code=APP_ICON&file_path=/appstore/app/icon/20110308/6_148x148.

KeyboardInterrupt: 